In [8]:
import pandas as pd
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
import ast
import numpy as np
import os
import ast
import urllib.request
from urllib.request import urlopen
from bs4 import BeautifulSoup
import os.path
from datetime import datetime
from collections import Counter

nltk.download('stopwords')
nltk.download('words')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/nefarion/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package words to /home/nefarion/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package wordnet to /home/nefarion/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [9]:
os.chdir("/mnt/f/Linda/Work work/Categorization")

input_file_relative = "Datasets/Feature_dataset.csv"
output_file_relative = "Datasets/hopefully_the_end.csv"
month = "October"
char_blacklist = list(chr(i) for i in range(32, 127) if i <= 64 or i >= 91 and i <= 96 or i >= 123)
stopwords = nltk.corpus.stopwords.words('english')
stopwords.extend(char_blacklist)
english_vocab = set(w.lower() for w in nltk.corpus.words.words())
english_tolerance = 50
english_confidence = []
words_threshold = 10
top = 2500
toker = RegexpTokenizer(r'((?<=[^\w\s])\w(?=[^\w\s])|(\W))+', gaps=True)
words_frequency = {}

In [10]:
# Read new generated data set file
df = pd.read_csv(input_file_relative)


In [11]:
top = 2500
words_frequency = {}
for category in set(df['main_category'].values):
    print(category)
    all_words = []
    for row in df[df['main_category'] == category]['tokens_en'].tolist():
        for word in ast.literal_eval(row):
            all_words.append(word)
    most_common = nltk.FreqDist(w for w in all_words).most_common(top)
    words_frequency[category] = most_common

Career_and_Education
Porn
HostingInternet_and_Telecom
Shopping
Web_Catalogue
Contractor
Guns
Skimming
Gift_Cards
Science
Crypto_Currency
Computer_and_Electronics
Finance
Drugs
Internet_and_Telecom
Autos_and_Vehicles
News_and_Media


In [12]:
words_frequency

{'Career_and_Education': [('copyright', 1), ('right', 1), ('reserved', 1)],
 'Porn': [('image', 23),
  ('child', 19),
  ('copy', 14),
  ('site', 11),
  ('material', 9),
  ('linked', 9),
  ('get', 8),
  ('account', 7),
  ('incest', 7),
  ('add', 7),
  ('login', 6),
  ('best', 5),
  ('order', 5),
  ('access', 5),
  ('content', 5),
  ('sexually', 5),
  ('explicit', 5),
  ('full', 5),
  ('new', 4),
  ('want', 4),
  ('use', 4),
  ('legal', 4),
  ('markdown', 4),
  ('category', 4),
  ('hour', 4),
  ('least', 3),
  ('month', 3),
  ('next', 3),
  ('tabu', 3),
  ('picture', 3),
  ('password', 3),
  ('personal', 3),
  ('adult', 3),
  ('right', 3),
  ('family', 3),
  ('brother', 3),
  ('sister', 3),
  ('link', 3),
  ('agreement', 3),
  ('medium', 3),
  ('thumbnail', 3),
  ('select', 3),
  ('take', 3),
  ('edit', 3),
  ('complete', 3),
  ('error', 3),
  ('minute', 3),
  ('day', 3),
  ('pure', 2),
  ('hacked', 2),
  ('provide', 2),
  ('high', 2),
  ('delivery', 2),
  ('two', 2),
  ('payment', 2),
 

In [13]:
for category in set(df['main_category'].values):
    words_frequency[category] = [word for word, number in words_frequency[category]]

In [14]:
from collections import Counter

import pprint
features = np.zeros(df.shape[0] * top).reshape(df.shape[0], top)
labels = np.zeros(df.shape[0])
counter = 0
for i, row in df.iterrows():
    c = [word for word, word_count in Counter(ast.literal_eval(row['tokens_en'])).most_common(top)]
    labels[counter] = list(set(df['main_category'].values)).index(row['main_category'])
    for word in c:
        if word in words_frequency[row['main_category']]:
            features[counter][words_frequency[row['main_category']].index(word)] = 1
    counter += 1

In [15]:
from sklearn.metrics import accuracy_score
from scipy.sparse import coo_matrix
X_sparse = coo_matrix(features)

from sklearn.utils import shuffle
X, X_sparse, y = shuffle(features, X_sparse, labels, random_state=0)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(X_train, y_train)
predictions = lr.predict(X_test)
score = lr.score(X_test, y_test)
print('LogisticRegression')
print('Score: ', score)
print('Top: ', top)
print('Dataset length: ', df.shape[0])
print()

from sklearn.svm import LinearSVC
clf = LinearSVC()
clf.fit(X_train, y_train)
predictions = clf.predict(X_test)
score = clf.score(X_test, y_test)
print('SVM')
print('Score: ', score)
print('Top: ', top)
print('Dataset length: ', df.shape[0])

LogisticRegression
Score:  0.17647058823529413
Top:  2500
Dataset length:  49

SVM
Score:  0.17647058823529413
Top:  2500
Dataset length:  49


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [16]:
import pandas as pd
import nltk
import ast
import numpy as np
import os
import ast
import urllib.request
from urllib.request import urlopen
from bs4 import BeautifulSoup
import os.path
nltk.download('stopwords')
nltk.download('words')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/nefarion/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package words to /home/nefarion/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package punkt to /home/nefarion/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Dataset creation if it is not existing.
__Dataset is filtered by these set of rules:__
1. Main category != Not_working (Exclude non working URL's)
2. Main category:confidence > 0.5 (Leave url's with likely know categories)
3. Non responding URL's are excluded
4. Non english language URL's are excluded.

### Caution, the full data set creation may take ~15 hours.

In [17]:
def no_filter_data():
    file = input_file_relative
    df = pd.read_csv(file)[['main_category', 'main_category:confidence', 'url']]
    df = df[(df['main_category'] != 'Not_working') & (df['main_category:confidence'] > 0.5)]
    df['tokens_en'] = ''
    
    counter = 0
    for i, row in df.iterrows():
        counter += 1
        url = row['url']
        print("{}, {}/{}".format(url, counter, len(df)))

        try:
            hdr = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
               'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
               'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
               'Accept-Encoding': 'none',
               'Accept-Language': 'en-US,en;q=0.8',
               'Connection': 'keep-alive'}
            req = urllib.request.Request(url, headers=hdr)
            html = urlopen(req).read()
#             html = urlopen('http://' + row['url'], timeout=15).read()
        except Exception as inst:
            print(f" {url} FAILED. because of {inst}")
            continue

        soup = BeautifulSoup(html, "html.parser")
        [tag.decompose() for tag in soup("script")]
        [tag.decompose() for tag in soup("style")]
        text = soup.get_text()
        lines = (line.strip() for line in text.splitlines())
        chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
        text = '\n'.join(chunk.lower() for chunk in chunks if chunk)
        tokens = nltk.word_tokenize(text)

        df.at[i, 'tokens_en'] = tokens if len(tokens) > 0 else ''
        
    df = df[~df['tokens_en'].isnull()]
    df.to_csv(output_file_relative)
    
    no_filter_data()

### Reading data set and creating list of stopwords and english vocabulary for further investigation

In [19]:
df = pd.read_csv(input_file_relative)
df = df[~df['tokens_en'].isnull()]
char_blacklist = list(chr(i) for i in range(32, 127) if i <= 64 or i >= 91 and i <= 96 or i >= 123)
stopwords = nltk.corpus.stopwords.words('english')
stopwords.extend(char_blacklist)
english_vocab = set(w.lower() for w in nltk.corpus.words.words())

# Filter webpages with english language
If the webpage contains at least 20 % english words of total words, then the webpage is considered as english

In [20]:
english_confidence = []
english_tolerance = 50
for i, row in df.iterrows():
    english_words = 0
    words = ast.literal_eval(row['tokens_en'])
    for word in words:
        if word.lower() in english_vocab:
            english_words += 1
    english_confidence.append(english_words / len(words) * 100)
df['english:confidence'] = english_confidence
df = df[df['english:confidence'] > english_tolerance]

# Make the most popular word list for each catgegory

In [21]:
top = 2500
words_frequency = {}
for category in set(df['main_category'].values):
    all_words = []
    for row in df[df['main_category'] == category]['tokens_en'].tolist():
        for word in ast.literal_eval(row):
            all_words.append(word)
                
    allWordExceptStopDist = nltk.FreqDist(
        w.lower() for w in all_words if w not in stopwords and len(w) >= 3 and w[0] not in char_blacklist)

    most_common = allWordExceptStopDist.most_common(top)
    words_frequency[category] = most_common

for category in set(df['main_category'].values):
    words_frequency[category] = [word for word, number in words_frequency[category]]

### Remove most frequent words in all categories

In [22]:
from collections import Counter
words = []
for category in words_frequency.keys():
    words.extend(words_frequency[category][0:15])
words_counter = Counter(words)
words_filter = {x : words_counter[x] for x in words_counter if words_counter[x] >= 7}
words_stop = list(words_filter.keys())
for category in words_frequency.keys():
    words_frequency[category] = [word for word in words_frequency[category] if word not in words_stop]

In [23]:
words_filter

{}

# Create features and labels for Machine learning training

In [24]:
from collections import Counter

features = np.zeros(df.shape[0] * top).reshape(df.shape[0], top)
labels = np.zeros(df.shape[0])
counter = 0
for i, row in df.iterrows():
    c = [word for word, word_count in Counter(ast.literal_eval(row['tokens_en'])).most_common(top)]
    labels[counter] = list(set(df['main_category'].values)).index(row['main_category'])
    for word in c:
        if word in words_frequency[row['main_category']]:
            features[counter][words_frequency[row['main_category']].index(word)] = 1
    counter += 1

# Create seperate training/testing datasets and shuffle them

In [25]:
from sklearn.metrics import accuracy_score
from scipy.sparse import coo_matrix
X_sparse = coo_matrix(features)

from sklearn.utils import shuffle
X, X_sparse, y = shuffle(features, X_sparse, labels, random_state=0)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

print('features____________________________________________:')
pprint.pprint(features)
print('labels____________________________________________:')
pprint.pprint(labels)

# Predictions

In [26]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(X_train, y_train)
predictions = lr.predict(X_test)
score = lr.score(X_test, y_test)
print('LogisticRegression')
print('Score: ', score)
print('Top: ', top)
print('Tolerance: ', english_tolerance)
print('Dataset length: ', df.shape[0])
print()
from sklearn.tree import DecisionTreeClassifier
dtc = DecisionTreeClassifier()
dtc.fit(X_train, y_train)
predictions = dtc.predict(X_test)
score = dtc.score(X_test, y_test)
print('DecisionTreeClassifier')
print('Score: ', score)
print('Top: ', top)
print('Tolerance: ', english_tolerance)
print('Dataset length: ', df.shape[0])
print()
from sklearn.svm import LinearSVC
clf = LinearSVC()
clf.fit(X_train, y_train) 
predictions = clf.predict(X_test)
score = clf.score(X_test, y_test)
print('SVM')
print('Score: ', score)
print('Top: ', top)
print('Tolerance: ', english_tolerance)
print('Dataset length: ', df.shape[0])

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression
Score:  0.11764705882352941
Top:  2500
Tolerance:  50
Dataset length:  49

DecisionTreeClassifier
Score:  0.058823529411764705
Top:  2500
Tolerance:  50
Dataset length:  49

SVM
Score:  0.11764705882352941
Top:  2500
Tolerance:  50
Dataset length:  49


/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


### Save ML model

In [19]:
month = 'October'
from sklearn.externals import joblib
filename = "Models/{}/LR_model_{}.joblib".format(month, month)
joblib.dump(lr, filename) 

import pickle
words_filename = "Models/{}/word_frequency_{}.picle".format(month, month)
pickle_out = open(words_filename,"wb")
pickle.dump(words_frequency, pickle_out)
pickle_out.close()
    
filename = "Models/{}/LR_maxtrain_{}.joblib".format(month, month)
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(X, y)
joblib.dump(lr, filename) 

/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


FileNotFoundError: [Errno 2] No such file or directory: 'Models/December/LR_model_v3_stop_December.joblib'

In [20]:
# import matplotlib.pyplot as plt; plt.rcdefaults()
# import numpy as np
# import matplotlib.pyplot as plt
 
# objects = ('English', 'Italic', 'Russian', 'Japan', 'China', 'Belgium')
# y_pos = np.arange(len(objects))
# performance = [8143,260,646,338,125,100]
 
# plt.bar(y_pos, performance, align='center', alpha=0.5)
# plt.xticks(y_pos, objects)
# plt.ylabel('URLs')
# plt.title('Languages diversity in the data set')
 
# plt.show()
# plt.savefig("language_diversity.png")
# df[df['main_category'] == 'Business_and_Industry']['url']

In [21]:
# import matplotlib.pyplot as plt; plt.rcdefaults()
# import numpy as np
# import matplotlib.pyplot as plt
# from collections import Counter

# words = []
# for category in words_frequency.keys():
#     words.extend(words_frequency[category][0:15])
# words_counter = Counter(words)
# words_filter = {x : words_counter[x] for x in words_counter if words_counter[x] >= 7}
# objects = tuple(words_filter.keys())
# y_pos = np.arange(len(objects))
# performance = list(words_filter.values())

# plt.barh(y_pos, performance, align='center', alpha=1)
# plt.xticks(range(1, max(performance) + 1))
# plt.yticks(y_pos, objects)
# plt.xlabel('Word diversity in categories (TOP 15 words)')
# plt.title('Words diversity in each category TOP 15 most frequent words')
 
# plt.show()
# plt.savefig("words_diversity.png")